Das Modul `Chess.jl` für die Sprache Julia stellt eine Implementation des Spiels Schach bereit.

In [1]:
using Chess

[TODO] Chess.jl erklären + Funktionen

## Schach-Brett

Mithilfe des `startboard()` Befehls lässt sich ein neues Schachbrett `Board` erzeugen, welches bereits alle Spielfiguren in deren Startpositionen bringt. Alternativ lässt sich ein Brett mit einem FEN-String (Forsyth-Edwards-Notation) über den Befehl `fromfen(String)` erstellen. [TODO] Fen generell

Innerhalb eines Jupyter-Notebooks wird ein `Board` direkt als Grafik dargestellt.

In [2]:
startboard()

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

In [3]:
fromfen("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

## Schach-Spiel

Mithilfe des `Game` Typs lässt sich ein Schach-Spiel erzeugen, welches neben einem statischen Brett über alle weiteren Funktionen und Mechaniken zum Spielen eines Schach-Spiels verfügt. Diese umfassen beispielsweise eine Zug-Historie, das Durchführen von Zügen oder deren Validierung.

In [4]:
game = SimpleGame()

SimpleGame:
 *

## Spielzüge

Ein Zug wird mithilfe des Datentyps `Move` dargestellt, welcher die Start- und Endposition umfasst. Beim Überprüfen des Zugs wird automatisch die zugrundeliegende Spielfigur inferiert, sodass diese nicht Teil des Structs ist.

Die einzelnen Felder des Schachbretts werden durch vordefinierte Konstanten gegeben, welche sich in ihrer Nomenklatur aus `SQ_` (Square) und den entsprechenden Feldkoordinaten, wie zb. `E2` zusammen: `SQ_E2`.

Möchte der Spieler beispielsweise als Eröffnung den Bauer von E2 auf E4 verschieben, so wird zunächst ein `Move` erzeugt: 

In [5]:
move = Move(SQ_E2, SQ_E4)

Move(e2e4)

Anschließend lässt sich dieser mithilfe `domove(Game; Move)` Befehls auf ein gegebendes Spiel anwenden:

In [6]:
domove!(game, move)

SimpleGame:
 1. e4 *

Um diesen Prozess zu vereinfachen, lässt sich die folgende Funktion definieren:

In [7]:
function mv(move::Move)
    domove!(game, move)
    return game
end

function mv(from::Square, to::Square)
    return mv(Move(from, to))
end

mv (generic function with 2 methods)

## Gegner: KI

Um ein tatsächliches Spiel durchführen zu können, wird ein Spielgegner benötigt: Unsere KI.

Diese wird zunächst so programmiert, dass mit jedem Aufruf ein zufälliger Zug durchführt wird.

Mithilfe des `moves(Board)` Befehls lassen sich alle möglichen Züge zu einer gegebenen Figurenanordnung auflisten:

In [8]:
moves(board(game))

20-element MoveList:
 Move(a7a6)
 Move(b7b6)
 Move(c7c6)
 Move(d7d6)
 Move(e7e6)
 Move(f7f6)
 Move(g7g6)
 Move(h7h6)
 Move(a7a5)
 Move(b7b5)
 Move(c7c5)
 Move(d7d5)
 Move(e7e5)
 Move(f7f5)
 Move(g7g5)
 Move(h7h5)
 Move(b8a6)
 Move(b8c6)
 Move(g8f6)
 Move(g8h6)

Der `rand(x)` Befehl selektiert ein zufälliges Element aus einer Auflistung:

In [9]:
rand(moves(board(game)))

Move(f7f6)

Aus diesem Komponenten lässt sich nun eine Funktion definieren, welche das Ziehen für die KI übernimmt:

In [10]:
function ai_mv()
    move = rand(moves(board(game)))
    domove!(game, move)
    return game
end

ai_mv()

SimpleGame:
 1. e4 e5 *

Die KI hat ihren ersten Zug gemacht! Mit den definierten Funktionen lässt sich nun bereits ein ganzes Schachspiel gegen eine naive "KI" spielen. Über die Chess.jl-Funtionen `isdraw(Game)` bzw. `ischeckmate(Game)` kann überprüft werden, ob es sich bei dem aktuellen Spielstatus um ein Unentschieden, bzw. ein Schachmatt handelt. Eine weitere Abstraktion bietet `isterminal(Game)` welche ausgibt, ob das aktuelle Spiel geendet hat (Schachmatt oder Unentschieden).

Durch das Zusammenfügen dieser Komponenten lässt sich bereits ein interaktives Schachspiel implementieren:

In [12]:
game = SimpleGame()
html = IJulia.HTML(IJulia.html(game))
IJulia.display(html)

ai_play = false # move to chess.jl lib
while !isterminal(game)
    if ai_play
        ai_mv()
        IJulia.clear_output()
        html = IJulia.HTML(IJulia.html(game))
        IJulia.display(html)
    else
        userinput = readline(stdin)
        if userinput == "exit"
           break 
        end
        move = movefromstring(userinput) # todo errors & parsing
        # todo add try-catch and ask user again
        mv(move)
    end
    ai_play = !ai_play
end

HTML{String}("SimpleGame:<div class=\"game\"><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"2\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wn.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"2\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wb.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"3\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"3\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bk.svg\" y=\"0\" height=\"1\" width=\"1\"></image></

stdin> exit
